In [1]:
from misc import *
from sklearn.preprocessing import LabelEncoder

import pymc3 as pm
import pandas as pd
import numpy as np


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Anaconda3\envs\MSR\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "C:\Anaconda3\envs\MSR\lib\site-packages\theano\gpuarray\__init__.py", line 227, in <module>
    use(config.device)
  File "C:\Anaconda3\envs\MSR\lib\

In [2]:
#Load the survey data

index1 = preprocess(pd.read_excel('data/RawData/IN10001.xls'))
index4 = preprocess(pd.read_excel('data/RawData/IN10004.xls'))
index5 = preprocess(pd.read_excel('data/RawData/IN10005.xls'))
index6 = preprocess(pd.read_excel('data/RawData/IN10006.xls'))
index7 = preprocess(pd.read_excel('data/RawData/IN10007.xls'))

#Create a new dataframe that only contains the demographic information that's included in each survey

common_columns = list(set(index1.columns).intersection(set(index7.columns)))

common_data = pd.concat([index1[common_columns],
                         index4[common_columns],
                         index5[common_columns],
                         index6[common_columns],
                         index7[common_columns]
                        ])

#Drop the columns that don't contain demographic information

common_data.drop(['ID', 'ADID IDFA', 'Time Started', 'Time Finished'], axis=1, inplace=True)

#Get count of respondents by state

common_data.groupby(['Area']).size().reset_index().values


array([['Alabama', 122],
       ['Alaska', 3],
       ['Arizona', 110],
       ['Arkansas', 37],
       ['California', 536],
       ['Colorado', 52],
       ['Connecticut', 44],
       ['Delaware', 9],
       ['District of Columbia', 33],
       ['Florida', 380],
       ['Georgia', 196],
       ['Hawaii', 10],
       ['Idaho', 22],
       ['Illinois', 286],
       ['Indiana', 105],
       ['Iowa', 59],
       ['Kansas', 36],
       ['Kentucky', 154],
       ['Louisiana', 72],
       ['Maine', 23],
       ['Maryland', 112],
       ['Massachusetts', 68],
       ['Michigan', 279],
       ['Minnesota', 70],
       ['Mississippi', 47],
       ['Missouri', 111],
       ['Montana', 9],
       ['Nebraska', 39],
       ['Nevada', 36],
       ['New Hampshire', 14],
       ['New Jersey', 96],
       ['New Mexico', 13],
       ['New York', 305],
       ['North Carolina', 220],
       ['North Dakota', 4],
       ['Ohio', 331],
       ['Oklahoma', 77],
       ['Oregon', 57],
       ['Pennsylvania', 

In [3]:
common_data.columns

Index(['Gender', 'DMA Name', 'US Census Region', 'Employment Status',
       'Postal Code', 'Productivity Booster',
       'Who will you vote for in the House of Representatives in 2018?',
       'US Congressional District', 'Musicfan', 'Marital Status', 'Insurance',
       'US Census Division', 'Car Purchase', 'Race', 'OS', 'Bookworm',
       'Socialite', 'Income', 'Area', 'Sportsfan',
       'How do you feel about the job Donald Trump is doing as president?',
       'Traveler', 'Number of children',
       'What is your political party affiliation?', 'Provider',
       'Food and Dining Lover', 'Year Of Birth', 'Age', 'Fashionista', 'Gamer',
       'Career', 'City', 'Job Seeker', 'DMA Code', 'Real Estate',
       'Value Shopper', 'Education', 'Entertainment Enthusiast'],
      dtype='object')

In [48]:
#Encode whether or not a respodent will vote in the 2018 House of Representatives election

def will_vote(answer):
    if answer == "Won't Vote":
        return 0
    else:
        return 1

#Encode whether or not a respondent will vote for a major party candidate given that he/she will be voting
    
def will_vote_major(answer):
    if answer == 'Will vote Republican' or answer == 'Will vote Democratic':
        return 1
    else:
        return 0

#Encode which major party candidate a respondent will vote for given that he/she will be voting for a major party candidate
    
def which_major(answer):
    if answer == 'Will vote Republican':
        return 0
    elif answer == 'Will vote Democratic':
        return 1
    
index1['Will Vote'] = index1['Who will you vote for in the House of Representatives in 2018?'].apply(lambda row: will_vote(row))
index1['Will Vote Major'] = index1['Who will you vote for in the House of Representatives in 2018?'].apply(lambda row: will_vote_major(row))
index1['Which Major'] = index1['Who will you vote for in the House of Representatives in 2018?'].apply(lambda row: which_major(row))


In [43]:
#Encode the gender, race, age, education, and state categories as integers

le = LabelEncoder()

index1['Gender Encoded'] = le.fit_transform(index1['Gender'])
index1['Race Encoded'] = le.fit_transform(index1['Race'])
index1['Age Encoded'] = le.fit_transform(index1['Age'])
index1['Area Encoded'] = le.fit_transform(index1['Area'])
index1['Education Encoded'] = le.fit_transform(index1['Education'])

#Create new columns for the marginal categories

index1['Race_Gender'] = index1['Race'].str.cat(index1['Gender'], sep='|')
index1['Age_Education'] = index1['Age'].str.cat(index1['Education'], sep='|')
index1['Age_Gender'] = index1['Age'].str.cat(index1['Gender'], sep='|')
index1['Education_Gender'] = index1['Education'].str.cat(index1['Gender'], sep='|')

#Encode the marginal categories as integers

index1['Race_Gender'] = le.fit_transform(index1['Race_Gender'])
index1['Age_Education'] = le.fit_transform(index1['Age_Education'])
index1['Age_Gender'] = le.fit_transform(index1['Age_Gender'])
index1['Education_Gender'] = le.fit_transform(index1['Education_Gender'])


In [49]:
#Group by race, gender, education, and age to see how many are in each cell and how many in each cell will vote

index1_unique = index1.groupby(['Race', 'Gender', 'Education', 'Age', 'Area'])['Will Vote']
index1_unique = index1_unique.agg([('Will Vote', 'sum'), ('n', 'size')]).reset_index()


In [49]:
(3*index1_unique.Gender + index1_unique.Race).values.size

637